# **Deep transfer learning based on MMD (Maximum Mean Discrepency)**
---
<font color=black size=3 face=雅黑>**Envirionment**: torch = 1.2.0 </font>

<font color=black size=3 face=雅黑>**A schematic introduction of implementation proposed feature-based models**</font>

<font color=black size=2 face=雅黑>In what follows, we explain how to implement a feature-based (MMD-CNN) in Pytorch for DTL-IFD

Schematically, the designed MMD-CNN looks like this:

* A `SharedNet` network maps vectors of shape `1024x1(64x64x3)` to extracted feature.
    
* A `Classifier` network maps source domain CNN feature to a Probability output with ten categories`(10,)`.
    
* A `rbf_MMD` function calculates the MMD loss between the source and target feature extracted from `SharedNet`.

    
*Step 1* : We first fed both source domain and target domain data 1D_CNN (2D_CNN) into the `SharedNet`, thus two types of features are 
extracted as `Source feature` and `Target feature`.

*Step 2* : We calculate the **MMD-Loss** between the `Source feature` and `Target feature`, the rbf-mmd-loss is defined as:
    $\operatorname{MMD}[\mathcal{F}, X, Y]=\left[\frac{1}{m^{2}} \sum_{i, j=1}^{m} k\left(x_{i}, x_{j}\right)-\frac{2}{m n} \sum_{i, j=1}^{m, n} k\left(x_{i}, y_{j}\right)+\frac{1}{n^{2}} \sum_{i, j=1}^{n} k\left(y_{i}, y_{j}\right)\right]^{\frac{1}{2}}$

*Step 3* : We calculate the **Classification-loss** only on the `Source feature`.
    
*Step 4* : We combine the **MMD-Loss** and **Classification-loss** together as **total-loss** and use this **total-loss** for backpropagation and optimize the `SharedNet` and `Classifier`.

**By iteratively conducting step 1 to step 4, the domain shift could be covered and the classfication task from source domain to target domain could be implemented** </font>

# Loading data and make the data-loader

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import time
import torch
import torch.nn as nn
from torch.optim import Adam
import torch.utils.data as Data
import pandas as pd
import math
from torchsummary import summary
import torch.nn.functional as F
from torch.autograd import Variable
import os
import scipy.io as sio

In [2]:
# Load 1D raw signal
X_0_1D = np.load("./DE/X_0.npy").reshape((-1,1,1024))
X_1_1D = np.load("./DE/X_1.npy").reshape((-1,1,1024))
X_2_1D = np.load("./DE/X_2.npy").reshape((-1,1,1024))
X_3_1D = np.load("./DE/X_3.npy").reshape((-1,1,1024))
# Load 2D pre-processed frequenct spectrum
X_0_2D = np.load("./DE/Case1_FFT.npy").reshape((-1,3,64,64))
X_1_2D = np.load("./DE/Case2_FFT.npy").reshape((-1,3,64,64))
X_2_2D = np.load("./DE/Case3_FFT.npy").reshape((-1,3,64,64))
X_3_2D = np.load("./DE/Case4_FFT.npy").reshape((-1,3,64,64))
Y_0 = np.load("./DE/Y_0.npy")
Y_1 = np.load("./DE/Y_1.npy")
Y_2 = np.load("./DE/Y_2.npy")
Y_3 = np.load("./DE/Y_3.npy")

# Defination of MMD

# **Defination of MMD (Maximum Mean Discrepency) and kernel function**
The basic MMD is defined as:
$M M D(\boldsymbol{X}, \boldsymbol{Y})=\left\|\sum_{i=1}^{n_{1}} \phi\left(x_{i}\right)-\sum_{j=1}^{n_{2}} \phi\left(y_{i}\right)\right\|_{\mathcal{H}}^{2}$

where the kernel function $\phi(x): \mathcal{X} \rightarrow \mathcal{H}$ indicates the mapping relationship

In [3]:
def guassian_kernel(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    n_samples = int(source.size()[0])+int(target.size()[0])
    total = torch.cat([source, target], dim=0)
    total0 = total.unsqueeze(0).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    total1 = total.unsqueeze(1).expand(int(total.size(0)), int(total.size(0)), int(total.size(1)))
    L2_distance = ((total0-total1)**2).sum(2)
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [torch.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]
    return sum(kernel_val)#/len(kernel_val)

def mmd_rbf_accelerate(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
        kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    loss = 0
    for i in range(batch_size):
        s1, s2 = i, (i+1)%batch_size
        t1, t2 = s1+batch_size, s2+batch_size
        loss += kernels[s1, s2] + kernels[t1, t2]
        loss -= kernels[s1, t2] + kernels[s2, t1]
    return loss / float(batch_size)
def mmd_rbf_noaccelerate(source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    batch_size = int(source.size()[0])
    kernels = guassian_kernel(source, target,
                              kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    XX = kernels[:batch_size, :batch_size]
    YY = kernels[batch_size:, batch_size:]
    XY = kernels[:batch_size, batch_size:]
    YX = kernels[batch_size:, :batch_size]
    loss = torch.mean(XX + YY - XY -YX)
    return loss

# Defination of the MMD-CNN model

## Defination MMD-CNN model for 2D data

In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class BaseNet_2D(nn.Module):

    def __init__(self, in_channel=1, out_channel=10):
        super(BaseNet_2D, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=5, stride=2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, padding=1),
            nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=5, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=5, stride=2))

    def forward(self, x):
        #x = self.Laplace(x)
        #self.featuremap_Laplace = x.detach() # 核心代码
        x = self.conv1(x)
        self.featuremap_conv1 = x.detach() # 核心代码
        x = self.conv2(x)
        self.featuremap_conv2 = x.detach() # 核心代码
        x = self.conv3(x)
        self.featuremap_conv3 = x.detach() # 核心代码
        x = self.conv4(x)
        self.featuremap_conv4 = x.detach() # 核心代码
        x = x.view(x.size(0), -1)
        return x
    
class CNN_2D(nn.Module):

    def __init__(self, num_classes=10):
        super(CNN_2D, self).__init__()
        self.sharedNet = BaseNet_2D()
        self.fc1 = nn.Linear(256 *11*11,1024)
        self.cls_fc = nn.Linear(1024, 10)

    def forward(self, source, target):
        loss = 0
        source = self.sharedNet(source)  
        if self.training == True:
            target = self.sharedNet(target)      
            #loss += mmd.mmd_rbf_accelerate(source, target)
            loss += mmd_rbf_noaccelerate(source, target)
        source = self.fc1(source)     
        source = self.cls_fc(source)
        #target = self.cls_fc(target)

        return source, loss

## Defination MMD-CNN model for 1D data

In [5]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from math import pi
def Laplace(p):
    A = 0.08
    ep = 0.03
    tal = 0.1
    f = 50
    w = 2 * pi * f
    q = torch.tensor(1 - pow(ep, 2))
    y = A * torch.exp((-ep / (torch.sqrt(q))) * (w * (p - tal))) * (-torch.sin(w * (p - tal)))
    return y


class Laplace_fast(nn.Module):

    def __init__(self, out_channels, kernel_size, in_channels=1):

        super(Laplace_fast, self).__init__()

        if in_channels != 1:

            msg = "MexhConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size - 1

        if kernel_size % 2 == 0:
            self.kernel_size = self.kernel_size + 1

        self.a_ = nn.Parameter(torch.linspace(1, 10, out_channels)).view(-1, 1)

        self.b_ = nn.Parameter(torch.linspace(0, 10, out_channels)).view(-1, 1)

    def forward(self, waveforms):

        time_disc = torch.linspace(0, 1, steps=int((self.kernel_size)))

        p1 = time_disc.cuda() - self.b_.cuda() / self.a_.cuda()

        laplace_filter = Laplace(p1)

        self.filters = (laplace_filter).view(self.out_channels, 1, self.kernel_size).cuda()


        return F.conv1d(waveforms, self.filters, stride=1, padding=1, dilation=1, bias=None, groups=1)




class BaseNet_1D(nn.Module):

    def __init__(self, in_channel=1, out_channel=10):
        super(BaseNet_1D, self).__init__()
        self.Laplace = nn.Sequential(
            Laplace_fast(64, 32),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2))
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(64, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=5, stride=2))
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True))
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 256, kernel_size=5, padding=1),
            nn.ReLU(inplace=True))
        self.conv4 = nn.Sequential(
            nn.Conv1d(256, 256, kernel_size=5, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=5, stride=2))

    def forward(self, x):
        x = self.Laplace(x)
        self.featuremap_Laplace = x.detach() # 核心代码
        x = self.conv1(x)
        self.featuremap_conv1 = x.detach() # 核心代码
        x = self.conv2(x)
        self.featuremap_conv2 = x.detach() # 核心代码
        x = self.conv3(x)
        self.featuremap_conv3 = x.detach() # 核心代码
        x = self.conv4(x)
        self.featuremap_conv4 = x.detach() # 核心代码
        x = x.view(x.size(0), -1)
        return x
    
class CNN_1D(nn.Module):

    def __init__(self, num_classes=10):
        super(CNN_1D, self).__init__()
        self.sharedNet = BaseNet_1D()
        self.fc1 = nn.Linear(256 * 120,1024)
        self.cls_fc = nn.Linear(1024, 10)

    def forward(self, source, target):
        loss = 0
        source = self.sharedNet(source)  
        if self.training == True:
            target = self.sharedNet(target)      
            #loss += mmd.mmd_rbf_accelerate(source, target)
            loss += mmd_rbf_noaccelerate(source, target)
        source = self.fc1(source)     
        source = self.cls_fc(source)
        #target = self.cls_fc(target)

        return source, loss

# Defination of the training process

In [6]:
def train(model):
    src_iter = iter(src_train_loader)
    tgt_iter = iter(tgt_train_loader)
    correct = 0
    best_model_wts=copy.deepcopy(model.state_dict())
    final_model_wts=copy.deepcopy(model.state_dict())
    for i in range(1, iteration+1):
        model.train()
        LEARNING_RATE = lr / math.pow((1 + 10 * (i - 1) / (iteration)), 0.75)
        if (i-1)%100==0:
            print('learning rate{: .4f}'.format(LEARNING_RATE) )
        optimizer = torch.optim.SGD([
        {'params': model.sharedNet.parameters()},
        {'params': model.cls_fc.parameters(), 'lr': LEARNING_RATE},
        ], lr=LEARNING_RATE / 10, momentum=momentum, weight_decay=l2_decay)
        try:
            src_data, src_label = src_iter.next()
        except Exception as err:
            src_iter=iter(src_train_loader)
            src_data, src_label = src_iter.next()
            
        try:
            tgt_data, _ = tgt_iter.next()
        except Exception as err:
            tgt_iter=iter(tgt_train_loader)
            tgt_data, _ = tgt_iter.next()
            
        if cuda:
            src_data, src_label = src_data.cuda(), src_label.cuda()
            tgt_data = tgt_data.cuda()

        optimizer.zero_grad()
        src_pred, mmd_loss = model(src_data, tgt_data)
        cls_loss = F.nll_loss(F.log_softmax(src_pred, dim=1), src_label.long())
        lambd = 0.1/ (1 + math.exp(-10 * (i) / iteration)) - 1
        loss = cls_loss + lambd * (mmd_loss)
        loss.backward()
        optimizer.step()
        if i % log_interval == 0:
            print('Train iter: {} [({:.0f}%)]\tLoss: {:.6f}\tsoft_Loss: {:.6f}\tmmd_Loss: {:.6f}'.format(
                i, 100. * i / iteration, loss.item(), cls_loss.item(), mmd_loss.item()))

        if i%(log_interval*20)==0:
            t_correct = test(model)
            if t_correct > correct:
                correct = t_correct
                best_model_wts=copy.deepcopy(model.state_dict())
            print('src: {} to tgt: {} max correct: {} max accuracy{: .2f}%\n'.format(
              src_name, tgt_name, correct, 100. * correct / tgt_dataset_len ))
        #if i>=1999:
            #final_model_wts=copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_wts)
    return model

# Defination of the testing process

In [7]:
def test(model):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for tgt_test_data, tgt_test_label in tgt_test_loader:
            if cuda:
                tgt_test_data, tgt_test_label = tgt_test_data.cuda(), tgt_test_label.cuda()
                tgt_test_data, tgt_test_label = Variable(tgt_test_data), Variable(tgt_test_label)
                tgt_pred, mmd_loss = model(tgt_test_data, tgt_test_data)
                test_loss += F.nll_loss(F.log_softmax(tgt_pred, dim = 1), tgt_test_label.long(), reduction='sum').item() # sum up batch loss
                pred = tgt_pred.data.max(1)[1] # get the index of the max log-probability
                correct += pred.eq(tgt_test_label.long().data.view_as(pred)).cpu().sum()

    test_loss /= tgt_dataset_len
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        tgt_name, test_loss, correct, tgt_dataset_len,
        100. * correct / tgt_dataset_len))
    return correct

# Model training

In [8]:
#Transform the 1D  data into tensor
X_0_1D=torch.from_numpy(X_0_1D.astype(np.float32))

X_1_1D=torch.from_numpy(X_1_1D.astype(np.float32))

X_2_1D=torch.from_numpy(X_2_1D.astype(np.float32))

X_3_1D=torch.from_numpy(X_3_1D.astype(np.float32))

#Transform the 2D  data into tensor
X_0_2D=torch.from_numpy(X_0_2D.astype(np.float32))

X_1_2D=torch.from_numpy(X_1_2D.astype(np.float32))

X_2_2D=torch.from_numpy(X_2_2D.astype(np.float32))

X_3_2D=torch.from_numpy(X_3_2D.astype(np.float32))

#Transform the  label into tensor
Y_0=torch.from_numpy(Y_0.astype(np.float32))

Y_1=torch.from_numpy(Y_1.astype(np.float32))

Y_2=torch.from_numpy(Y_2.astype(np.float32))

Y_3=torch.from_numpy(Y_3.astype(np.float32))

#Combine the 1D-X and Y with TensorDataset
data_0_1D=Data.TensorDataset(X_0_1D,Y_0)
data_1_1D=Data.TensorDataset(X_1_1D,Y_1)
data_2_1D=Data.TensorDataset(X_2_1D,Y_2)
data_3_1D=Data.TensorDataset(X_3_1D,Y_3)

#Combine the 2D-X and Y with TensorDataset
data_0_2D=Data.TensorDataset(X_0_2D,Y_0)
data_1_2D=Data.TensorDataset(X_1_2D,Y_1)
data_2_2D=Data.TensorDataset(X_2_2D,Y_2)
data_3_2D=Data.TensorDataset(X_3_2D,Y_3)


def Split(full_dataset):
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    return train_dataset, test_dataset

# 1D training data and testing data 
Train_0_1D,Test_0_1D = Split(data_0_1D)
Train_1_1D,Test_1_1D = Split(data_1_1D)
Train_2_1D,Test_2_1D = Split(data_2_1D)
Train_3_1D,Test_3_1D = Split(data_3_1D)

# 2D training data and testing data 
Train_0_2D,Test_0_2D = Split(data_0_2D)
Train_1_2D,Test_1_2D = Split(data_1_2D)
Train_2_2D,Test_2_2D = Split(data_2_2D)
Train_3_2D,Test_3_2D = Split(data_3_2D)
#Defination of data loader
src_train_loader=Data.DataLoader(dataset=Train_0_2D,batch_size=16,shuffle=True,num_workers=0)
tgt_train_loader=Data.DataLoader(dataset=Train_1_2D,batch_size=16,shuffle=True,num_workers=0)
src_test_loader=Data.DataLoader(dataset=Test_0_2D,batch_size=16,shuffle=True,num_workers=0)
tgt_test_loader=Data.DataLoader(dataset=Test_1_2D,batch_size=16,shuffle=True,num_workers=0)

In [9]:

# Training settings
batch_size =16
iteration=1000
lr = 0.001
momentum = 0.9
no_cuda =False
seed = 8
log_interval = 10
l2_decay = 5e-4
model = CNN_2D().cuda()
print(model)
src_dataset_len = len(src_train_loader.dataset)
tgt_dataset_len = len(tgt_test_loader.dataset)
src_loader_len = len(src_train_loader)
tgt_loader_len = len(tgt_train_loader)
src_name = "a"
tgt_name = "b"
no_cuda =False
cuda = not no_cuda and torch.cuda.is_available()
if cuda:
    model.cuda()
my_net=train(model)

CNN_2D(
  (sharedNet): BaseNet_2D(
    (conv1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv4): Sequential(
      (0): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=5, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (fc1): Linear(in_features=30976, out_features=1024, bias=True)
  (cls_fc): Linear(in_features=1024, out_features=10, bias=True)
)
learning rate 0.0010
Train iter: 10 [(1%)]	Loss: 2.463044	soft_Loss: 2.892381	mmd_Loss: 0.453125
Trai

Train iter: 860 [(86%)]	Loss: -0.123386	soft_Loss: 0.199539	mmd_Loss: 0.358799
Train iter: 870 [(87%)]	Loss: -0.363580	soft_Loss: 0.148151	mmd_Loss: 0.568579
Train iter: 880 [(88%)]	Loss: -0.576198	soft_Loss: 0.090255	mmd_Loss: 0.740492
Train iter: 890 [(89%)]	Loss: -0.572154	soft_Loss: 0.091364	mmd_Loss: 0.737231
Train iter: 900 [(90%)]	Loss: -0.269056	soft_Loss: 0.154372	mmd_Loss: 0.470469
learning rate 0.0002
Train iter: 910 [(91%)]	Loss: -0.524307	soft_Loss: 0.126250	mmd_Loss: 0.722832
Train iter: 920 [(92%)]	Loss: -0.233430	soft_Loss: 0.094341	mmd_Loss: 0.364186
Train iter: 930 [(93%)]	Loss: -0.418766	soft_Loss: 0.148872	mmd_Loss: 0.630702
Train iter: 940 [(94%)]	Loss: -0.191008	soft_Loss: 0.094864	mmd_Loss: 0.317633
Train iter: 950 [(95%)]	Loss: -0.399662	soft_Loss: 0.076095	mmd_Loss: 0.528615
Train iter: 960 [(96%)]	Loss: -0.463621	soft_Loss: 0.098067	mmd_Loss: 0.624094
Train iter: 970 [(97%)]	Loss: -0.160582	soft_Loss: 0.113562	mmd_Loss: 0.304603
Train iter: 980 [(98%)]	Loss: -

# Model saving

In [10]:
my_net.eval()
torch.save(my_net, 'Feature-based-Task1-2D.pkl')

C:\Users\phoen\Anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN_2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\phoen\Anaconda3\envs\pytorch\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type BaseNet_2D. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# Model evaluation

## Model loading 

In [11]:
model1 = torch.load('Feature-based-Task1-2D.pkl')

## Model testing

In [13]:
#Defination of data loader
src_train_loader=Data.DataLoader(dataset=Train_0_2D,batch_size=16,shuffle=True,num_workers=0)
tgt_train_loader=Data.DataLoader(dataset=Train_1_2D,batch_size=16,shuffle=True,num_workers=0)
src_test_loader=Data.DataLoader(dataset=Test_0_2D,batch_size=16,shuffle=True,num_workers=0)
tgt_test_loader=Data.DataLoader(dataset=Test_1_2D,batch_size=16,shuffle=True,num_workers=0)
model1.eval()
test_loss = 0
correct = 0
src_dataset_len = len(src_train_loader.dataset)
tgt_dataset_len = len(tgt_test_loader.dataset)
src_loader_len = len(src_train_loader)
tgt_loader_len = len(tgt_train_loader)
total=[]
for tgt_test_data, tgt_test_label in tgt_test_loader:
    no_cuda =False
    cuda = not no_cuda and torch.cuda.is_available()
    if cuda:
        
        tgt_test_data, tgt_test_label = tgt_test_data.cuda(), tgt_test_label.cuda()
        tgt_test_data, tgt_test_label = Variable(tgt_test_data), Variable(tgt_test_label)
        tgt_pred, mmd_loss = model1(tgt_test_data, tgt_test_data)
        test_loss += F.nll_loss(F.log_softmax(tgt_pred, dim = 1), tgt_test_label.long(), reduction='sum').item() # sum up batch loss
        pred = tgt_pred.data.max(1)[1] # get the index of the max log-probability
        correct_each=pred.eq(tgt_test_label.long().data.view_as(pred)).cpu().sum()/ torch.tensor(tgt_dataset_len//10).float()
        correct_each=correct_each.numpy()
        total.append(correct_each)
        std=np.std(total)
        correct += pred.eq(tgt_test_label.long().data.view_as(pred)).cpu().sum()
        
print(' meab accuracy{: .2f}%\n and mean std {: .2f}%\n'.format(100* correct / torch.tensor(tgt_dataset_len).float(),100*std ))

 meab accuracy 96.80%
 and mean std  0.34%

